**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Check the params of the kube-apiserver

Add `PodSecurityPolicy` if necessary. 
The server will automatically restart on saving the file

In [1]:
grep -E '$|PodSecurityPolicy|RBAC' --color=always /etc/kubernetes/manifests/kube-apiserver.yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubeadm.kubernetes.io/kube-apiserver.advertise-address.endpoint: 192.168.49.2:8443
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-apiserver
    - --advertise-address=192.168.49.2
    - --allow-privileged=true
    - --authorization-mode=Node,RBAC
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
    - --etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
    - --etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
    - --insecure-port

Verify the process is running with appropriate parameters

In [ ]:
kubectl apply -f https://k8s.io/examples/application/deployment.yaml

In [ ]:
kubectl get all

## Everything works, we are fine
### let's destroy the shiny, happyworld

In [ ]:
kubectl delete deployment nginx-deployment

In [ ]:
sed -i 's/--enable-admission-plugins=.*$/&,PodSecurityPolicy/' /etc/kubernetes/manifests/kube-apiserver.yaml

In [2]:
grep -E '$|PodSecurityPolicy|RBAC' --color=always /etc/kubernetes/manifests/kube-apiserver.yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubeadm.kubernetes.io/kube-apiserver.advertise-address.endpoint: 192.168.49.2:8443
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-apiserver
    - --advertise-address=192.168.49.2
    - --allow-privileged=true
    - --authorization-mode=Node,RBAC
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota,PodSecurityPolicy
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
    - --etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
    - --etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
   

In [3]:
ps auxwww | grep  -E "kube-apiserver|PodSecurityPolicy" --color=always |  sed  "s/ --/\n--/g"  

root       22229 14.6  2.0 1097348 318096 ?      Ssl  11:17   0:11 kube-apiserver
--advertise-address=192.168.49.2
--allow-privileged=true
--authorization-mode=Node,RBAC
--client-ca-file=/var/lib/minikube/certs/ca.crt
--enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,MutatingAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota,PodSecurityPolicy
--enable-bootstrap-token-auth=true
--etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
--etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
--etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
--etcd-servers=https://127.0.0.1:2379
--insecure-port=0
--kubelet-client-certificate=/var/lib/minikube/certs/apiserver-kubelet-client.crt
--kubelet-client-key=/var/lib/minikube/certs/apiserver-kubelet-client.key
--kubelet-preferred-address-types=InternalIP,ExternalIP,Hostname
--proxy-client-cert-file=/var/lib/minikube/certs/front-proxy-client.crt

## Check the influence of empty PodSecurityPolicies

Can we start an nginx?

In [12]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: default
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
EOF

deployment.apps/nginx-deployment created


In [15]:
kubectl get all

NAME                                          READY   STATUS    RESTARTS   AGE
pod/nginx-with-token-again-66868cff8b-7llg6   1/1     Running   0          35m

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   81m

NAME                                     READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment         0/1     0            0           20s
deployment.apps/nginx-with-token-again   1/1     1            1           36m

NAME                                                DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-6799fc88d8         1         0         0       20s
replicaset.apps/nginx-with-token-again-66868cff8b   1         1         1       35m
replicaset.apps/nginx-with-token-again-7d7cd958c    0         0         0       36m


In [16]:
RS=$(kubectl get rs | awk '/nginx-deployment/{print $1}' | head -1)
echo $RS

nginx-deployment-6799fc88d8


In [17]:
kubectl describe rs $RS | grep --color=always -E '$|FailedCreate|forbidden' 

Name:           nginx-deployment-6799fc88d8
Namespace:      default
Selector:       app=nginx,pod-template-hash=6799fc88d8
Labels:         app=nginx
                pod-template-hash=6799fc88d8
Annotations:    deployment.kubernetes.io/desired-replicas: 1
                deployment.kubernetes.io/max-replicas: 2
                deployment.kubernetes.io/revision: 1
Controlled By:  Deployment/nginx-deployment
Replicas:       0 current / 1 desired
Pods Status:    0 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  app=nginx
           pod-template-hash=6799fc88d8
  Containers:
   nginx:
    Image:        nginx
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type             Status  Reason
  ----             ------  ------
  ReplicaFailure   True    FailedCreate
Events:
  Type     Reason        Age                From                   Message
  ----     ------        ----          

Without an existing PSP nobody can start anything. 

Note that the *replicaset* is reporting the failure.

The pod does not exist, thus there are no logs we can watch

In [18]:
kubectl get psp

NAME          PRIV    CAPS   SELINUX    RUNASUSER   FSGROUP    SUPGROUP   READONLYROOTFS   VOLUMES
restrictive   false   *      RunAsAny   RunAsAny    RunAsAny   RunAsAny   false            configMap,downwardAPI,emptyDir,persistentVolumeClaim,secret,projected


# Policies

* can be applied to each account, including 
 * Users
 * ServiceAccounts
* give fine grained control over rights

We start with the *restrictive* Policy from the Kubernets Documentation
https://kubernetes.io/docs/concepts/policy/pod-security-policy


In [10]:
kubectl create -f  - <<EOF
apiVersion: policy/v1beta1
kind: PodSecurityPolicy
metadata:
  name: restrictive
spec:
  privileged: false
  hostNetwork: false
  allowPrivilegeEscalation: false
  defaultAllowPrivilegeEscalation: false
  hostPID: false
  hostIPC: false
  runAsUser:
    rule: RunAsAny
  fsGroup:
    rule: RunAsAny
  seLinux:
    rule: RunAsAny
  supplementalGroups:
    rule: RunAsAny
  volumes:
  - 'configMap'
  - 'downwardAPI'
  - 'emptyDir'
  - 'persistentVolumeClaim'
  - 'secret'
  - 'projected'
  allowedCapabilities:
  - '*'
EOF

podsecuritypolicy.policy/restrictive created


In [11]:
kubectl delete deployment nginx-deployment

deployment.apps "nginx-deployment" deleted


In [19]:
kubectl create -f- <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: pause
spec:
  containers:
    - name: pause
      image: k8s.gcr.io/pause
EOF

pod/pause created


In [21]:
kubectl get pods

NAME                                      READY   STATUS    RESTARTS   AGE
nginx-with-token-again-66868cff8b-7llg6   1/1     Running   0          37m
pause                                     1/1     Running   0          8s


In [ ]:
kubectl get pod $(kubectl get po | egrep -o nginx[A-Za-z0-9-]+) -o yaml | grep -i psp

## Again


Go back to the nginx example and start again
Let us restart the deployment again

Still not working because the psp is not bound

## Roles 

We create a restrictive *ClusterRole*

In [22]:
kubectl apply -f - <<EOF
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: psp-restrictive
rules:
- apiGroups:
  - extensions
  resources:
  - podsecuritypolicies
  resourceNames:
  - restrictive
  verbs:
  - use
EOF

clusterrole.rbac.authorization.k8s.io/psp-restrictive created


## RoleBindings

We need to bind the role to a user

In [23]:
kubectl create -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: restrictive
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: psp-restrictive
subjects:
- kind: ServiceAccount
  name: default
  namespace: default
EOF

rolebinding.rbac.authorization.k8s.io/restrictive created


In [28]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: default
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx
EOF

deployment.apps/nginx-deployment created


In [31]:
kubectl get all

NAME                                          READY   STATUS    RESTARTS   AGE
pod/nginx-deployment-6799fc88d8-znnnz         1/1     Running   0          14s
pod/nginx-with-token-again-66868cff8b-7llg6   1/1     Running   0          40m
pod/pause                                     1/1     Running   0          3m2s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   87m

NAME                                     READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx-deployment         1/1     1            1           15s
deployment.apps/nginx-with-token-again   1/1     1            1           41m

NAME                                                DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-deployment-6799fc88d8         1         1         1       15s
replicaset.apps/nginx-with-token-again-66868cff8b   1         1         1       40m
replicaset.apps/nginx-with-token-again-7d7cd95

## Create a Permissive Policy

## Which policy has been applied

how can we detect, which policy is in use for a given pod 

In [ ]:
POD=$(kubectl get  pod | awk '/nginx-deployment/{print $1}' | head -1)
echo $POD

In [ ]:
kubectl get pods

Pedestrians approach

In [ ]:
kubectl get pod $(kubectl get po | egrep -o nginx[A-Za-z0-9-]+) -o yaml | grep -i psp

Enjoy the beauty of the jsonpath template,

`.metadata.annotations` is a map

In [ ]:
kubectl get pod $POD -o jsonpath='{.metadata.annotations}'

`kubernetes\.io/psp` is a valid name
* the `/` is a usual character in a string
* the `.` needs to be escaped

In [ ]:
kubectl get pod $POD -o jsonpath='{.metadata.annotations.kubernetes\.io/psp}'

and now all pods

In [ ]:
kubectl get pod --all-namespaces -o jsonpath='{.items[*].metadata.annotations.kubernetes\.io/psp}'

In [ ]:
kubectl get psp

In [ ]:
kubectl delete psp restrictive

In [ ]:
kubectl delete deployment nginx-deployment

In [ ]:
kubect 

In [ ]:
kubectl create -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-strict
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF


In [ ]:
kubectl get pods

In [ ]:
kubectl delete deployment nginx-strict

## Summary 

* we have started one pod
* with the restrictive PodSecurityPolicy
* bound to the default serviceaccount
* in the default namespace

[ ![High Five](https://image.freepik.com/free-photo/blonde-african-young-women-giving-high-five-against-grey-background_23-2148178786.jpg) ](https://www.freepik.com/free-photo/blonde-african-young-women-giving-high-five-against-grey-background_4732025.htm)

Attribution: [Background photo created by freepik](https://www.freepik.com/photos/background), thanks to [freepik](https://www.freepik.com)